### Imports

In [100]:
import pandas as pd
import numpy as np

### Data loading

In [101]:
path = "../Bloomberg_data_processing/"
filename = "preprocessed_sp500_final_data_01-02-22.csv"

data = pd.read_csv(path + filename)

In [102]:
data.head(2)

,Ticker,Security,GICS Sector,GICS Sub-Industry,CUR_MKT_CAP,CURR_ENTP_VAL,NET_INCOME_FIRM,SALES_REV_TURN,NET_REV,GROSS_PROFIT,EBIT,EBITDA,NET_DEBT,CURRENT_EV_TO_T12M_EBIT,CURRENT_EV_TO_T12M_EBITDA,PX_TO_SALES_RATIO,TRAIL_12M_NET_SALES,CURRENT_EV_TO_12M_SALES,PE_RATIO
0,MMM,3M,Industrials,Industrial Conglomerates,9.566949e+10,108530.49040,5809.107585,35355.0,NaN,16560.0,7369.0,9420.0,12861.0,14.686868,NaN,2.718985,35355.0,3.069735,16.427379
1,AOS,A. O. Smith,Industrials,Building Products,1.216317e+10,11750.76783,490.448034,3538.9,NaN,1310.9,609.5,687.4,-412.4,19.226403,17.094512,3.459942,3538.9,3.320458,25.857856


### GICS sectors & sub-industry data exploration

In [103]:
data['GICS Sector'].nunique()

11

In [104]:
data.groupby(['GICS Sector'])["GICS Sector"].count()

GICS Sector
Communication Services    27
Consumer Discretionary    60
Consumer Staples          32
Energy                    21
Financials                67
Health Care               64
Industrials               73
Information Technology    75
Materials                 28
Real Estate               29
Utilities                 29
Name: GICS Sector, dtype: int64

In [105]:
data["GICS Sub-Industry"].nunique()

124

In [106]:
data["GICS Sub-Industry"].describe()

count                       505
unique                      124
top       Health Care Equipment
freq                         19
Name: GICS Sub-Industry, dtype: object

In [107]:
pd.DataFrame(data.groupby(['GICS Sector', 'GICS Sub-Industry'])["GICS Sub-Industry"].count()).head(15)

GICS Sub-Industry
GICS Sector            GICS Sub-Industry                                       
Communication Services Advertising                                            2
                       Alternative Carriers                                   1
                       Broadcasting                                           2
                       Cable & Satellite                                      3
                       Integrated Telecommunication Services                  2
                       Interactive Home Entertainment                         3
                       Interactive Media & Services                           5
                       Movies & Entertainment                                 6
                       Publishing                                             2
                       Wireless Telecommunication Services                    1
Consumer Discretionary Apparel Retail                                         2
                       Apparel, Accessories & Luxury Goods                    7
                       Auto Parts & Equipment                                 2
                       Automobile Manufacturers                               3
                       Automotive Retail                                      1

### Companies similarity ranking

Similarity will be determined based on sector, sub-industry, geography, and market cap (in this order of importance).

In [108]:
# API à tester pour récupérer le nom du pays à partir de la 'Headquarters location' :
# https://apilayer.com/marketplace/description/geo-api#pricing

In [109]:
# dataset with only usefull columns
companies_relevant_info = data[["Ticker", "GICS Sector", "GICS Sub-Industry", "CUR_MKT_CAP"]].copy()
companies_relevant_info.head()

,Ticker,GICS Sector,GICS Sub-Industry,CUR_MKT_CAP
0,MMM,Industrials,Industrial Conglomerates,9.566949e+10
1,AOS,Industrials,Building Products,1.216317e+10
2,ABT,Health Care,Health Care Equipment,2.253860e+11
3,ABBV,Health Care,Pharmaceuticals,2.420050e+11
4,ABMD,Health Care,Health Care Equipment,1.346134e+10


In [110]:
companies_relevant_info["CUR_MKT_CAP"] = np.log(companies_relevant_info["CUR_MKT_CAP"])
companies_relevant_info["CUR_MKT_CAP"] = companies_relevant_info["CUR_MKT_CAP"] - companies_relevant_info["CUR_MKT_CAP"].min()
companies_relevant_info["CUR_MKT_CAP"] = companies_relevant_info["CUR_MKT_CAP"] / companies_relevant_info["CUR_MKT_CAP"].max()
companies_relevant_info.describe()

,CUR_MKT_CAP
count,502.000000
mean,0.288228
std,0.169697
min,0.000000
25%,0.168170
50%,0.261567
75%,0.376664
max,1.000000


In [111]:
print("Number of missing values per column :\n")
print(companies_relevant_info.isnull().sum())

Number of missing values per column :

Ticker               0
GICS Sector          0
GICS Sub-Industry    0
CUR_MKT_CAP          3
dtype: int64


In [112]:
# getting the studied company's data
studied_comp_ticker = "MMM"
studied_comp_data = companies_relevant_info.loc[companies_relevant_info["Ticker"] == studied_comp_ticker].iloc[0]
studied_comp_data

Ticker                                    MMM
GICS Sector                       Industrials
GICS Sub-Industry    Industrial Conglomerates
CUR_MKT_CAP                          0.439908
Name: 0, dtype: object

In [113]:
# computing distance between the studied company and every other company
# similarity_scores = []
similarity_scores = pd.DataFrame(index=companies_relevant_info["Ticker"].copy())
similarity_scores["similarity_score"] = 0.0

for index, row in companies_relevant_info.iterrows():
    similarity_score = 0 # lowest = most similar
    if(row["GICS Sector"] != studied_comp_data["GICS Sector"]):
        similarity_score += 10 # biggest penalty, sector is most important thing the companies need to have in common
    elif(row["GICS Sub-Industry"] != studied_comp_data["GICS Sub-Industry"]):
        similarity_score += 1
    similarity_score += abs(row["CUR_MKT_CAP"] - studied_comp_data["CUR_MKT_CAP"])
    
    # todo : make geopgraphy be taken into account in the scoring
    # and if one info is missing (market cap mostly) either leave as NaN (will be considered lowest similarity and that company will never be used) or, if we want to use that company nevertheless, give penalty of either max or average difference for that info (for example max diff for mk cap is 1 and avg is 0.288228)

    similarity_scores["similarity_score"][row["Ticker"]] = similarity_score

similarity_scores
    


,similarity_score
Ticker,
MMM,0.000000
AOS,1.340261
ABT,10.141371
ABBV,10.153108
ABMD,10.323531
...,...
YUM,10.158106
ZBRA,10.207438
ZBH,10.216847


In [116]:
sorted_similarity_scores = similarity_scores["similarity_score"].sort_values()
sorted_similarity_scores.head(15)

Ticker
MMM    0.000000
GE     0.013380
HON    0.063717
ROP    0.120398
LMT    1.016871
CAT    1.021569
DE     1.031796
BA     1.032838
CSX    1.038190
ITW    1.043663
RTX    1.056806
NSC    1.062950
FDX    1.063397
ETN    1.068528
WM     1.069101
Name: similarity_score, dtype: float64